## Understanding resnet

In [41]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB ? eta -:--:--
   -------- ------------------------------- 1.6/7.8 MB 7.7 MB/s eta 0:00:01
   ------------- -------------------------- 2.6/7.8 MB 5.4 MB/s eta 0:00:01
   ------------------------ --------------- 4.7/7.8 MB 6.5 MB/s eta 0:00:01
   ---------------------------------- ----- 6.8/7.8 MB 7.4 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 7.4 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 10.3 MB/s eta 0:00:00
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)


In [7]:
import tensorflow as tf

from tensorflow.keras import datasets,layers,models,losses,Model

In [20]:
(x_train,y_train),(x_test,y_test)=datasets.mnist.load_data()


In [24]:
x_train=tf.pad(x_train,[[0,0],[2,2],[2,2]])/255
print(x_train.shape)
x_test=tf.pad(x_test,[[0,0],[2,2],[2,2]])/255
print(x_test.shape)

(60000, 32, 32)
(10000, 32, 32)


In [28]:
x_train=tf.expand_dims(x_train,axis=3 ,name=None)
x_test=tf.expand_dims(x_test,axis=3,name=None)
x_test = tf.image.grayscale_to_rgb(x_test)
x_train=tf.image.grayscale_to_rgb(x_train)


In [32]:
x_val = x_train[-2000:,:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:,:]
y_train = y_train[:-2000]

In [34]:
base_model = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
  layer.trainable = False

94765736/94765736 [==============================] - 11s 0us/step


In [36]:
x = layers.Flatten()(base_model.output)

x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

In [38]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [40]:
history = head_model.fit(x_train, y_train, batch_size=64, epochs=40, validation_data=(x_val, y_val))

Epoch 1/40
907/907 [==============================] - 45s 25ms/step - loss: 0.6085 - accuracy: 0.8096 - val_loss: 0.3115 - val_accuracy: 0.9065
Epoch 2/40
907/907 [==============================] - 20s 22ms/step - loss: 0.3479 - accuracy: 0.8877 - val_loss: 0.1988 - val_accuracy: 0.9415
Epoch 3/40
907/907 [==============================] - 20s 22ms/step - loss: 0.3076 - accuracy: 0.9001 - val_loss: 0.1940 - val_accuracy: 0.9415
Epoch 4/40
907/907 [==============================] - 20s 22ms/step - loss: 0.2796 - accuracy: 0.9081 - val_loss: 0.2219 - val_accuracy: 0.9340
Epoch 5/40
907/907 [==============================] - 20s 22ms/step - loss: 0.2542 - accuracy: 0.9164 - val_loss: 0.2144 - val_accuracy: 0.9345
Epoch 6/40
907/907 [==============================] - 20s 22ms/step - loss: 0.2407 - accuracy: 0.9216 - val_loss: 0.1916 - val_accuracy: 0.9430
Epoch 7/40
907/907 [==============================] - 20s 22ms/step - loss: 0.2296 - accuracy: 0.9243 - val_loss: 0.2304 - val_accuracy:

In [42]:
head_model.evaluate(x_test, y_test)

313/313 [==============================] - 7s 23ms/step - loss: 0.1287 - accuracy: 0.9591


[0.12868542969226837, 0.9591000080108643]

In [48]:
head_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              